In [1]:
import torch
from transformers import pipeline
from pprint import pprint
import pandas as pd
import polars as pl
import pydoc
from glob import glob

c:\Users\zanga\Documents\Projects\complaints_analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unique_companies = []
for file in glob(r'data\complaint_data_finance_hf\*'):
    print('Process_file: ', file)
    df = pl.read_parquet(file)
    df = df.select(pl.col("Company").unique())
    unique_companies.append(df)

Process_file:  data\complaint_data_finance_hf\0.parquet
Process_file:  data\complaint_data_finance_hf\1.parquet
Process_file:  data\complaint_data_finance_hf\2.parquet
Process_file:  data\complaint_data_finance_hf\3.parquet
Process_file:  data\complaint_data_finance_hf\4.parquet
Process_file:  data\complaint_data_finance_hf\5.parquet
Process_file:  data\complaint_data_finance_hf\6.parquet
Process_file:  data\complaint_data_finance_hf\7.parquet


In [3]:
unique_companies = pl.concat(unique_companies)
unique_companies = unique_companies.select(pl.col("Company").unique())

In [4]:
#unique_companies.write_excel(r'data_proc_steps\unique_companies\unique_companies.xlsx')

In [5]:
include_companies = pl.read_excel(r'data_proc_steps\unique_companies\unique_companies.xlsx')
include_companies = include_companies.filter(pl.col('Keep')==True)
include_companies = (
    include_companies
    .select(pl.col("Company"))
    .to_series()
    .to_list()
)

In [6]:
included_companies = []
for file in glob(r'data\complaint_data_finance_hf\*'):
    print('Process_file: ', file)
    df = pl.read_parquet(file)
    df = df.filter(pl.col("Company").is_in(include_companies))
    included_companies.append(df)
included_companies = pl.concat(included_companies)
included_companies = included_companies.filter(pl.col('Consumer complaint narrative').is_not_null())

Process_file:  data\complaint_data_finance_hf\0.parquet
Process_file:  data\complaint_data_finance_hf\1.parquet
Process_file:  data\complaint_data_finance_hf\2.parquet
Process_file:  data\complaint_data_finance_hf\3.parquet
Process_file:  data\complaint_data_finance_hf\4.parquet
Process_file:  data\complaint_data_finance_hf\5.parquet
Process_file:  data\complaint_data_finance_hf\6.parquet
Process_file:  data\complaint_data_finance_hf\7.parquet


In [7]:
included_companies = included_companies.select([
    "Date received",
    "Consumer complaint narrative",
    "Complaint ID",
    "Product",
    "Issue",
    "Sub-issue",
])
included_companies
#included_companies.write_parquet(r'data_proc_steps\training_data\clean_data.parquet')

Date received,Consumer complaint narrative,Complaint ID,Product,Issue,Sub-issue
str,str,str,str,str,str
"""2025-06-20""","""I am formally disputing inaccu…","""14185804""","""Credit reporting or other pers…","""Problem with a company's inves…","""Their investigation did not fi…"
"""2023-08-04""","""I am listing accounts below th…","""7353440""","""Credit reporting, credit repai…","""Problem with a credit reportin…","""Investigation took more than 3…"
"""2025-06-15""","""Wells Fargo keeps sending me r…","""14077536""","""Credit card""","""Problem with a purchase shown …","""Credit card company isn't reso…"
"""2023-06-05""","""In accordance with the Fair Cr…","""7072421""","""Credit reporting, credit repai…","""Problem with a credit reportin…","""Their investigation did not fi…"
"""2025-06-03""","""In accordance with the fair cr…","""13871672""","""Credit reporting or other pers…","""Improper use of your report""","""Reporting company used your re…"
…,…,…,…,…,…
"""2024-12-10""","""I bank at Chase Bank. On Thurs…","""11078583""","""Credit card""","""Trouble using your card""","""Can't use card to make purchas…"
"""2025-05-23""","""I affirm that the inclusion of…","""13693541""","""Credit reporting or other pers…","""Incorrect information on your …","""Account status incorrect"""
"""2017-02-24""","""BANK OF AMERICA OPENED AN ACCO…","""2357033""","""Consumer Loan""","""Managing the loan or lease""",null


In [8]:
included_companies.write_parquet(r'data_proc_steps\training_data\clean_data.parquet')

In [ ]:
import math

df = included_companies
n_splits = 8
chunk_size = math.ceil(df.height / n_splits)

for i in range(n_splits):
    start = i * chunk_size
    df_chunk = df.slice(start, chunk_size)
    
    out_path = fr"data_proc_steps\split_data\clean_data_part_{i+1}.parquet"
    df_chunk.write_parquet(out_path)
    
    print(f"Saved {out_path} with {df_chunk.height} rows")

    #hi

Saved data_proc_steps\split_data\clean_data_part_1.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_2.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_3.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_4.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_5.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_6.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_7.parquet with 84621 rows
Saved data_proc_steps\split_data\clean_data_part_8.parquet with 84616 rows
